# 1. Giới thiệu
Xử lý Ngôn ngữ Tự nhiên (NLP) là một lĩnh vực trong trí tuệ nhân tạo (AI) tập trung vào việc giúp máy tính hiểu và xử lý ngôn ngữ con người. NLP được sử dụng trong nhiều ứng dụng như dịch máy, trợ lý ảo (như Siri hoặc Alexa), hoặc phân tích cảm xúc từ các bài đánh giá trên mạng. Mục tiêu của NLP là giúp máy tính có thể đọc, hiểu và trả lời các câu hỏi bằng ngôn ngữ tự nhiên, giống như cách chúng ta giao tiếp với nhau.

Tuy nhiên, ngôn ngữ con người rất phức tạp. Mỗi từ có thể có nhiều nghĩa, và ý nghĩa của một từ có thể thay đổi tùy thuộc vào ngữ cảnh. Để máy tính hiểu được ngôn ngữ, chúng ta cần một cách để biểu diễn các từ một cách ý nghĩa và hiệu quả. Đó là nơi mà word embeddings (biểu diễn từ) xuất hiện.

## 1.1. Đặt vấn đề: Giới hạn của biểu diễn one-hot

Trong các tác vụ NLP, bước đầu tiên và quan trọng là **biểu diễn từ ngữ dưới dạng số** để đưa vào mô hình học máy. Một phương pháp trực quan và phổ biến là **vector one-hot**, trong đó mỗi từ được biểu diễn bằng một vector nhị phân có độ dài bằng kích thước từ điển, với đúng một phần tử bằng 1 và các phần tử còn lại bằng 0.

Mặc dù đơn giản và dễ triển khai, biểu diễn one-hot tồn tại những **hạn chế nghiêm trọng**:

- **Không có thông tin ngữ nghĩa**: Vector của “cat” và “dog” hoàn toàn trực giao, không phản ánh bất kỳ sự tương đồng nào giữa chúng.
- **Sparsity cao**: Độ dài vector bằng kích thước từ điển (có thể lên tới hàng triệu từ), dẫn đến tiêu tốn bộ nhớ và tính toán kém hiệu quả.
- **Không học được quan hệ ngữ cảnh**: Các vector không thể cập nhật để phản ánh mối quan hệ ngữ cảnh trong văn bản.

Vì vậy, cần có một phương pháp biểu diễn từ tốt hơn, vượt qua được các hạn chế trên


## 1.2. Giải pháp: Word Embedding

**Word embedding** là kỹ thuật học biểu diễn các từ dưới dạng vector thực (dense vector) có chiều thấp (ví dụ từ 50 đến 100, thấp hơn nhiều so với số từ khả dĩ trong từ điển của bất kỳ ngôn ngữ thông dụng nào), sao cho các vector này **mang thông tin ngữ nghĩa và ngữ cảnh**.

Khác với vector one-hot, các vector embedding được học từ dữ liệu, nhờ đó:
- Các từ có ý nghĩa gần nhau sẽ có vector gần nhau.
- Có thể thực hiện các phép toán hình học như:
  $$
  \text{vec}(\text{"king"}) - \text{vec}(\text{"man"}) + \text{vec}(\text{"woman"}) \approx \text{vec}(\text{"queen"})
  $$
- Được dùng như vector biểu diễn từ cho các bài toán xử lý ngôn ngữ tự nhiên khác như: phân loại văn bản, sinh văn bản, dịch máy, v.v.

Một trong những mô hình học embedding thành công và phổ biến nhất là **Word2Vec**, bao gồm hai biến thể chính: **Skip-Gram** và **CBOW**.


Word embedding không chỉ cải thiện khả năng xử lý ngôn ngữ của mô hình, mà còn:
- **Giảm đáng kể số chiều**
- **Tăng hiệu quả học**: Vector liên tục giúp mô hình tổng quát tốt hơn.
- **Cho phép học không giám sát**: Có thể huấn luyện embedding từ kho văn bản lớn mà không cần gán nhãn.


## 1.3. Ứng dụng của Word Embedding

Word embedding là nền tảng cốt lõi của hầu hết các mô hình NLP hiện đại. Một số ứng dụng tiêu biểu bao gồm:

- **Tìm kiếm ngữ nghĩa**: Gợi ý truy vấn hoặc tài liệu dựa trên từ tương đồng.
- **Phân loại văn bản**: Phân loại văn bản theo nhu cầu như phân loại email rác, phân loại xu hướng bình luận, ...
- **Dịch máy**: Embedding giúp mô hình hiểu nghĩa và ngữ pháp của từ.
- **Sinh văn bản**: Các mô hình như GPT sử dụng embedding làm lớp đầu vào


## 1.4. Mục tiêu của chương này

Trong chương này, chúng ta sẽ cùng tìm hiểu chi tiết mô hình **Skip-gram** và **CBOW**: cách xây dựng mô hình, các công thức toán học, và cách huấn luyện chúng

# 2. Word2vec

## 2.1. Giới thiệu

Word2vec là một kỹ thuật được giới thiệu bởi Tomas Mikolov vào năm 2013 để học word embeddings từ một lượng lớn dữ liệu văn bản. Word2vec không cần dữ liệu đã được gắn nhãn (như phân loại từ); thay vào đó, nó sử dụng self-supervised learning, nghĩa là nó tự học từ chính dữ liệu bằng cách dự đoán một phần dữ liệu dựa trên phần khác.

Word2vec có hai mô hình chính:
- **Skip-gram**: Skip-gram dự đoán các từ ngữ cảnh dựa trên từ mục tiêu. Với từ "sits", mô hình cố gắng dự đoán các từ như "the", "cat", "on", "the", "mat".
- **Continuous Bag of Words (CBOW)**: Ngược lại với Skip-gram, dự đoán một từ mục tiêu dựa trên các từ ngữ cảnh xung quanh nó. Ví dụ, với câu "the cat sits on the mat", CBOW lấy các từ "the", "cat", "on", "the", "mat" để dự đoán từ "sits".

Dưới đây là bảng so sánh nhanh giữa hai mô hình:

|Mô hình |	Mục tiêu |	Ví dụ|
|--------|-----------|-------|
|Skip-gram|	Dự đoán context từ center word|	Từ "ăn" dự đoán "tôi" và "táo".|
|Continuous Bag of Words (CBOW)|	Dự đoán center word từ context|	Từ "tôi" và "táo" dự đoán "ăn".|

<div style="text-align: center;">
    <div style="display: inline-block;">
        <p style="font-style: italic; color: gray; text-align: center; margin: 4px 0 0 0;">Bảng 1: So sánh Skip-gram và CBOW.</p>
    </div>
</div>

<div style="text-align: center;">
    <div style="display: inline-block;">
        <img src="https://github.com/hcmut-aa-math/word-embedding/blob/master/assets/pic3-cbow-and-skipgram.png?raw=true" style="width: 600px; height: auto;" />
        <p style="font-style: italic; color: gray; text-align: center; margin: 4px 0 0 0;">
            Hình 3: Kiến trúc CBOW dự đoán từ hiện tại dựa trên ngữ cảnh, còn Skip-gram dự đoán các từ xung quanh dựa vào từ hiện tại.
        </p>
    </div>
</div>

Cả hai mô hình đều dựa trên các công thức toán học để tính xác suất điều kiện, sử dụng hàm softmax và dot product giữa các vector từ. Dưới đây là phân tích chi tiết từng công thức.

## 2.2. Skip-gram
### 2.2.1 Định nghĩa
Mô hình skip-gam giả định rằng một từ có thể được sử dụng để sinh ra các từ xung quanh nó trong một chuỗi văn bản. Ví dụ, giả sử chuỗi văn bản là “the”, “man”, “loves”, “his” và “son”. Ta sử dụng “loves” làm từ đích trung tâm và đặt kích thước cửa sổ ngữ cảnh bằng 2. Với từ đích trung tâm “loves”, mô hình skip-gram quan tâm đến xác suất có điều kiện sinh ra các từ ngữ cảnh (“the”, “man”, “his” và “son”) nằm trong khoảng cách không quá 2 từ:

$$
P(\textrm{"the"},\textrm{"man"},\textrm{"his"},\textrm{"son"}\mid\textrm{"loves"}).
$$

Ta giả định rằng, với từ đích trung tâm cho trước, các từ ngữ cảnh được sinh ra độc lập với nhau. Trong trường hợp này, công thức trên có thể được viết lại thành

$$
P(\textrm{"the"}\mid\textrm{"loves"})\cdot P(\textrm{"man"}\mid\textrm{"loves"})\cdot P(\textrm{"his"}\mid\textrm{"loves"})\cdot P(\textrm{"son"}\mid\textrm{"loves"}).
$$


<div style="text-align: center;">
    <div style="display: inline-block;">
        <img src="https://github.com/hcmut-aa-math/word-embedding/blob/master/assets/pic4-skipgram.png?raw=true" style="width: 600px; height: auto;" />
        <p style="font-style: italic; color: gray; text-align: center; margin: 4px 0 0 0;">
            Hình 4: Mô hình skip-gram quan tâm đến xác suất có điều kiện sinh ra các từ ngữ cảnh với một từ đích trung tâm cho trước..
        </p>
    </div>
</div>


Trong mô hình skip-gam, mỗi từ được biểu diễn bằng hai vector $d-\text{chiều}$ (một dùng khi từ $w$ là từ ngữ cảnh, một dùng khi từ $w$ là từ trung tâm) để tính xác suất có điều kiện. Giả sử chỉ số của một từ trong từ điển là $i$, vector của từ được biểu diễn là $\mathbf{v}_i\in\mathbb{R}^d$ khi từ này là từ đích trung tâm và là $\mathbf{u}_i\in\mathbb{R}^d$ khi từ này là một từ ngữ cảnh. Gọi $c$ và $o$ lần lượt là chỉ số của từ đích trung tâm $w_c$ và từ ngữ cảnh $w_o$ trong từ điển. Có thể thu được xác suất có điều kiện sinh ra từ ngữ cảnh cho một từ đích trung tâm cho trước bằng phép toán softmax trên tích vô hướng của vector:

$$
P(w_o \mid w_c) = \frac{\exp(\mathbf{u}_o^\top \mathbf{v}_c)}{ \sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)},
$$

Trong đó:
- $w_c$: Từ trung tâm (center word), ví dụ "cat" trong câu "the cat sat on mat".
- $w_o$: Từ ngữ cảnh (output word), ví dụ "sat" trong cùng câu.
- $\mathbf{v}_c \in \mathbb{R}^d$: Vector biểu diễn của từ trung tâm $w_c$ (input vector).
- $\mathbf{u}_o \in \mathbb{R}^d$: Vector biểu diễn của từ ngữ cảnh $w_o$ (output vector).
- $\mathcal{V}$: Tập hợp tất cả từ trong từ điển (vocabulary). Tập chỉ số trong bộ từ vựng là $\mathcal{V} = \{0, 1, \ldots, |\mathcal{V}|-1\}$.
- $\mathbf{u}_i \in \mathbb{R}^d$: Vector biểu diễn của từ $i$ trong từ điển.
- $\exp$: Hàm mũ, $\exp(x) = e^x$
- $\mathbf{u}_o^\top \mathbf{v}_c$: Tích vô hướng (dot product) giữa vector $\mathbf{u}_o$ và $\mathbf{v}_c$, đo lường mức độ tương đồng giữa từ trung tâm và từ ngữ cảnh.
- $\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)$: Tổng chuẩn hóa (normalization term) giữa các điểm tương đồng mũ giữa từ trung tâm $w_c$ trên toàn bộ từ điển $\mathcal{V}$, đảm bảo các xác suất cộng lại bằng 1.

**Ví dụ**:

+ Dữ liệu và thiết lập
  - Câu mẫu: "The cat sits on the mat."
  - Từ điển $ \mathcal{V} $: Giả sử từ điển nhỏ gồm 5 từ: $ \mathcal{V} = \{\text{"the"}, \text{"cat"}, \text{"sits"}, \text{"on"}, \text{"mat"}\} $, $ |\mathcal{V}| = 5 $.
  - Chiều vector embedding: $ d = 2 $.
  - Cửa sổ ngữ cảnh: window size = 1.
  - Từ trung tâm $ w_c $: "cat".
  - Từ ngữ cảnh $ w_o $: "The" (trước 1 từ) và "sits" (sau 1 từ).

+ Vector embedding (giả định sau huấn luyện)
  - Giả sử sau khi huấn luyện mô hình skip-gram, các vector embedding đã được học như sau:

    - $ \mathbf{v}_{\text{cat}} = \mathbf{v}_c = [0.5, -0.2] $
    - $ \mathbf{u}_{\text{the}} = [0.3, 0.1] $
    - $ \mathbf{u}_{\text{sits}} = [0.4, -0.3] $
    - $ \mathbf{u}_{\text{on}} = [0.1, 0.8] $
    - $ \mathbf{u}_{\text{mat}} = [-0.2, 0.5] $
    - Giả sử $ \mathbf{u}_{\text{cat}} = [0.6, -0.1] $

+ Tính tích vô hướng $ \mathbf{u}_o^\top \mathbf{v}_c $

  - Với $ w_o = \text{"the"} $:
  $ \mathbf{u}_{\text{the}}^\top \mathbf{v}_{\text{cat}} $ = $ (0.3 \cdot 0.5) + (0.1 \cdot (-0.2)) = 0.15 - 0.02 = 0.13 $

  - Với $ w_o = \text{"sits"} $:
  $ \mathbf{u}_{\text{sits}}^\top \mathbf{v}_{\text{cat}} $ = $ (0.4 \cdot 0.5) + ((-0.3) \cdot (-0.2)) = 0.2 + 0.06 = 0.26 $

  - Với $ w_o = \text{"on"} $:
  $ \mathbf{u}_{\text{on}}^\top \mathbf{v}_{\text{cat}} $ = $ (0.1 \cdot 0.5) + (0.8 \cdot (-0.2)) = 0.05 - 0.16 = -0.11 $

  - Với $ w_o = \text{"mat"} $:
  $ \mathbf{u}_{\text{mat}}^\top \mathbf{v}_{\text{cat}} $ = $ ((-0.2) \cdot 0.5) + (0.5 \cdot (-0.2)) = -0.1 - 0.1 = -0.2 $

  - Với $ w_o = \text{"cat"} $ (tự so sánh với chính nó):
  $ \mathbf{u}_{\text{cat}}^\top \mathbf{v}_{\text{cat}} $ = $ (0.6 \cdot 0.5) + ((-0.1) \cdot (-0.2)) = 0.3 + 0.02 = 0.32 $

+ Tính giá trị mũ $ \exp(\mathbf{u}_o^\top \mathbf{v}_c) $ : Sử dụng hàm mũ $ \exp(x) = e^x $ (dùng $ e \approx 2.718 $)

  - $ \exp(0.13) \approx 1.138 $
  - $ \exp(0.26) \approx 1.297 $
  - $ \exp(-0.11) \approx 0.896 $
  - $ \exp(-0.2) \approx 0.819 $
  - $ \exp(0.32) \approx 1.377 $

+ Tính mẫu số (chuẩn hóa softmax) : Mẫu số là tổng các giá trị mũ trên tất cả các từ trong $ V $

  $ \sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c) $ = $ 1.138 + 1.297 + 0.896 + 0.819 + 1.377 \approx 5.527 $

+ Tính xác suất $ P(w_o | w_c) $ cho mỗi từ : Sử dụng công thức softmax

  - $ P(\text{"the"} | \text{"cat"}) = \frac{\exp(0.13)}{5.527} = \frac{1.138}{5.527} \approx 0.206 $
  - $ P(\text{"sits"} | \text{"cat"}) = \frac{\exp(0.26)}{5.527} = \frac{1.297}{5.527} \approx 0.235 $
  - $ P(\text{"on"} | \text{"cat"}) = \frac{\exp(-0.11)}{5.527} = \frac{0.896}{5.527} \approx 0.162 $
  - $ P(\text{"mat"} | \text{"cat"}) = \frac{\exp(-0.2)}{5.527} = \frac{0.819}{5.527} \approx 0.148 $
  - $ P(\text{"cat"} | \text{"cat"}) = \frac{\exp(0.32)}{5.527} = \frac{1.377}{5.527} \approx 0.249 $

+ Kiểm tra tổng xác suất

  $ 0.206 + 0.235 + 0.162 + 0.148 + 0.249 \approx 1.000 $

  (Kết quả gần 1 do làm tròn số, chứng tỏ công thức softmax hoạt động đúng).

+ Phân tích kết quả
  - Từ ngữ cảnh thực tế: Trong câu "The cat sits on the mat", "The" và "sits" là các từ ngữ cảnh của "cat". Xác suất cao nhất thuộc về "sits" (0.235) và "the" (0.206), phù hợp với dữ liệu.
  - Từ không liên quan: "on" (0.162) và "mat" (0.148) có xác suất thấp hơn, vì chúng nằm ngoài window size = 1 (cách "cat" 1 từ).
  - Tự so sánh: "cat" có xác suất 0.249 khi so với chính nó, cho thấy mô hình cũng học được mối quan hệ tự phản xạ.

+ Ý nghĩa
  - Ví dụ này minh họa cách công thức softmax sử dụng tích vô hướng $ \mathbf{u}_o^\top \mathbf{v}_c $ để đo lường mức độ tương thích giữa từ trung tâm và từ ngữ cảnh.
  - Giá trị mũ $ \exp(\cdot) $ khuếch đại sự khác biệt và chuẩn hóa đảm bảo tổng xác suất bằng 1.
  - Kết quả phản ánh đúng ngữ cảnh trong câu, cho thấy mô hình skip-gram học được mối quan hệ từ dữ liệu.

### 2.2.2. Hàm hợp lý (likelihood)
Giả sử ta có một chuỗi văn bản với độ dài $T$, trong đó từ tại vị trí $t$ được ký hiệu là $w^{(t)}$. Mô hình skip-gram giả định rằng:
- Các từ ngữ cảnh trong cửa sổ kích thước $m$ (bao gồm $m$ từ bên trái và $m$ từ bên phải của từ trung tâm) được sinh ra **độc lập** với nhau, khi biết từ đích trung tâm.
- Hàm hợp lý (likelihood) của mô hình là xác suất kết hợp để sinh ra tất cả các từ ngữ cảnh cho mọi từ trung tâm trong chuỗi văn bản.

Hàm hợp lý được định nghĩa như sau:

$$
L = \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)}),
$$

Nếu $t+j < 1$ hoặc $t+j > T$ (vị trí ngoài chuỗi văn bản), các số hạng này bị bỏ qua. Ví dụ, với $t=1$ và $m=2$, các vị trí $j=-2, -1$ sẽ không tồn tại và được bỏ qua.

Ý nghĩa của hàm hợp lý:
- Hàm $L$ biểu thị xác suất tổng quát để mô hình skip-gram sinh ra toàn bộ các cặp từ trung tâm và từ ngữ cảnh trong tập dữ liệu.
- Mục tiêu của MLE là tìm các tham số ($\mathbf{v}_i$, $\mathbf{u}_i$) sao cho $L$ đạt giá trị lớn nhất, tức là mô hình dự đoán các từ ngữ cảnh chính xác nhất có thể.


### 2.2.3. Trainning
Mục tiêu của skip-gram là cực đại hóa hàm hợp lý (Maximum Likelihood Estimation - MLE), tức là tìm các vector $\mathbf{v}_i$ và $\mathbf{u}_i$ để cho $L$ đạt giá trị lớn nhất. Hay mô hình có thể dự đoán chính xác các từ ngữ cảnh dựa trên từ trung tâm.

Trong thực tế, vì $L$ là một tích của nhiều xác suất nhỏ, ta thường làm việc với log-likelihood để tránh vấn đề số học (numerical underflow):
$$
\log L = \sum_{t=1}^T \sum_{-m \leq j \leq m, j \neq 0} \log P(w^{(t+j)} \mid w^{(t)}).
$$

Hàm mất mát được định nghĩa là phủ định của log-likelihood:
$$
J = -\log L = -\sum_{t=1}^T \sum_{-m \leq j \leq m, j \neq 0} \log P(w^{(t+j)} \mid w^{(t)}).
$$

Hàm mất mát này là tổng của logarit xác suất có điều kiện trên tất cả các cặp từ trung tâm và từ ngữ cảnh trong tập huấn luyện. Việc giảm thiểu hàm mất mát này giúp mô hình cải thiện khả năng dự đoán.

Cực đại hóa $\log L$ tương đương với giảm thiểu $J$, để huấn luyện, ta sử dụng [**Stochastic Gradient Descent (SGD)**](https://en.wikipedia.org/wiki/Stochastic_gradient_descent). Trong mỗi vòng lặp, ta lấy mẫu ngẫu nhiên một chuỗi con nhỏ hơn (mini-batch) từ tập dữ liệu, tính toán hàm mất mát cho chuỗi con đó, sau đó tính gradient để cập nhật các tham số mô hình.
#### 2.2.3.1. Tính toán gradient: Điểm then chốt của huấn luyện

Để cập nhật các vector $\mathbf{v}_c$ và $\mathbf{u}_o$, ta cần tính gradient của hàm mất mát đối với các tham số. Điểm mấu chốt là tính gradient của logarit xác suất có điều kiện $\log P(w_o \mid w_c)$, trong đó $w_c$ là từ đích trung tâm và $w_o$ là từ ngữ cảnh.

Theo định nghĩa trong mô hình skip-gram, xác suất có điều kiện được tính như sau:

$$
P(w_o \mid w_c) = \frac{\exp(\mathbf{u}_o^\top \mathbf{v}_c)}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)},
$$

và logarit của nó là:

$$
\begin{align*}
\log P(w_o \mid w_c)
&= \log \frac{\exp(\mathbf{u}_o^\top \mathbf{v}_c)}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)} \\
&= \log \exp(\mathbf{u}_o^\top \mathbf{v}_c) - \log\left(\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)\right) \\
&= \mathbf{u}_o^\top \mathbf{v}_c - \log\left(\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)\right)
\end{align*}
$$


#### 2.2.3.2. Chứng minh gradient theo $\mathbf{v}_c$

Ta cần tính đạo hàm của $\log P(w_o \mid w_c)$ theo $\mathbf{v}_c$:

$$
\begin{align*}
\frac{\partial}{\partial \mathbf{v}_c}\log P(w_o \mid w_c)
&= \frac{\partial}{\partial \mathbf{v}_c} \left( \mathbf{u}_o^\top \mathbf{v}_c - \log\left(\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)\right) \right) \\
&= \frac{\partial}{\partial \mathbf{v}_c} \mathbf{u}_o^\top \mathbf{v}_c - \frac{\partial}{\partial \mathbf{v}_c} \log\left(\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)\right) \\
&= \mathbf{u}_o - \frac{1}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)} \cdot \sum_{j \in \mathcal{V}} \exp(\mathbf{u}_j^\top \mathbf{v}_c) \mathbf{u}_j
\end{align*}
$$

Nhận thấy:
$$
\frac{\exp(\mathbf{u}_j^\top \mathbf{v}_c)}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)} = P(w_j \mid w_c),
$$
nên:
$$
\frac{1}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)} \cdot \sum_{j \in \mathcal{V}} \exp(\mathbf{u}_j^\top \mathbf{v}_c) \mathbf{u}_j = \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j.
$$
Do đó, gradient tổng quát là:
$$
\frac{\partial \log P(w_o \mid w_c)}{\partial \mathbf{v}_c} = \mathbf{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j.
$$
**Ý nghĩa**: Gradient này bao gồm hai phần:
- $\mathbf{u}_o$: Vector từ ngữ cảnh thực tế, đại diện cho từ $w_o$ mà mô hình cần dự đoán.
- $\sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j$: Kỳ vọng của vector từ ngữ cảnh trên toàn bộ từ điển, dựa trên xác suất dự đoán.

Gradient này được sử dụng để cập nhật $\mathbf{v}_c$ trong SGD theo công thức:
$$
\mathbf{v}_c \leftarrow \mathbf{v}_c + \eta \left( \mathbf{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j \right),
$$
với $\eta$ là tốc độ học (learning rate).
#### 2.2.3.3. Kết quả sau huấn luyện

Sau khi huấn luyện, với mỗi từ $w_i$ trong từ điển (có chỉ số $i$), ta thu được hai vector:

- $\mathbf{v}_i$: Vector từ đích trung tâm.
- $\mathbf{u}_i$: Vector từ ngữ cảnh.

Trong các ứng dụng NLP (như phân loại văn bản, phân tích cảm xúc), **vector từ đích trung tâm $\mathbf{v}_i$** thường được sử dụng làm biểu diễn chính cho từ $w_i$, vì nó được tối ưu hóa để dự đoán ngữ cảnh xung quanh.

#### 2.2.3.4 Ví dụ minh họa
##### 1. Thiết lập bài toán
Chuỗi văn bản: "the cat sits on mat".
Tập từ vựng: $\mathcal{V} = { \text{the}, \text{cat}, \text{sits}, \text{on}, \text{mat} }$, với kích thước từ vựng $|\mathcal{V}| = 5$.
Độ dài chuỗi: $T = 5$ (5 từ).
Kích thước cửa sổ: $m = 1$ (chỉ xét 1 từ bên trái và 1 từ bên phải của từ trung tâm).
Vector từ: Mỗi từ được biểu diễn bởi hai vector: $\mathbf{v}_i$ (vector từ trung tâm) và $\mathbf{u}_i$ (vector từ ngữ cảnh). Để đơn giản, giả sử mỗi vector có kích thước 2 (tức là $\mathbf{v}_i, \mathbf{u}_i \in \mathbb{R}^2$).
Tốc độ học: $\eta = 0.1$.

##### 2. Khởi tạo vector
Giả sử các vector từ được khởi tạo ngẫu nhiên như sau (các giá trị chỉ mang tính minh họa):



Từ
$\mathbf{v}_i$ (trung tâm)
$\mathbf{u}_i$ (ngữ cảnh)



the
$[0.1, 0.2]$
$[0.3, 0.4]$


cat
$[0.5, 0.6]$
$[0.7, 0.8]$


sits
$[0.9, 1.0]$
$[1.1, 1.2]$


on
$[1.3, 1.4]$
$[1.5, 1.6]$


mat
$[1.7, 1.8]$
$[1.9, 2.0]$


##### 3. Chọn một cặp từ trung tâm và từ ngữ cảnh
Ta xét từ trung tâm tại vị trí $t = 2$, tức là từ "cat", và từ ngữ cảnh tại $j = 1$, tức là từ "sits" (vì "sits" nằm ngay bên phải "cat" trong chuỗi).

Từ trung tâm: $w_c = \text{cat}$, với $\mathbf{v}_c = [0.5, 0.6]$.

Từ ngữ cảnh: $w_o = \text{sits}$, với $\mathbf{u}_o = [1.1, 1.2]$.

Ta cần tính:

Xác suất $P(w_o \mid w_c) = P(\text{sits} \mid \text{cat})$.

Log-likelihood $\log P(w_o \mid w_c)$.

Gradient $\frac{\partial \log P(w_o \mid w_c)}{\partial \mathbf{v}_c}$.

Cập nhật vector $\mathbf{v}_c$ bằng SGD.

##### 4. Tính xác suất $P(w_o \mid w_c)$
Xác suất được tính theo công thức softmax:
$$
P(w_o \mid w_c) = \frac{\exp(\mathbf{u}_o^\top \mathbf{v}c)}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)}.
$$
Bước 1: Tính tích vô hướng $\mathbf{u}_i^\top \mathbf{v}_c$

Vector $\mathbf{v}_c = [0.5, 0.6]$ (cho "cat"). Tính tích vô hướng với từng $\mathbf{u}_i$:

Với $w_o = \text{sits}$, $\mathbf{u}_o = [1.1, 1.2]$: $\mathbf{u}_o^\top \mathbf{v}_c = 1.1 \cdot 0.5 + 1.2 \cdot 0.6 = 0.55 + 0.72 = 1.27.$, $\exp(1.27) \approx 3.562.$

Với $w = \text{the}$, $\mathbf{u}_{\text{the}} = [0.3, 0.4]$:$\mathbf{u}_{\text{the}}^\top \mathbf{v}_c = 0.3 \cdot 0.5 + 0.4 \cdot 0.6 = 0.15 + 0.24 = 0.39.$, $\exp(0.39) \approx 1.477.$

Với $w = \text{cat}$, $\mathbf{u}_{\text{cat}} = [0.7, 0.8]$: $\mathbf{u}_{\text{cat}}^\top \mathbf{v}_c = 0.7 \cdot 0.5 + 0.8 \cdot 0.6 = 0.35 + 0.48 = 0.83.$, $[\exp(0.83) \approx 2.292.$

Với $w = \text{on}$, $\mathbf{u}_{\text{on}} = [1.5, 1.6]$:$\mathbf{u}_{\text{on}}^\top \mathbf{v}_c = 1.5 \cdot 0.5 + 1.6 \cdot 0.6 = 0.75 + 0.96 = 1.71.$, $[\exp(1.71) \approx 5.530.$

Với $w = \text{mat}$, $\mathbf{u}_{\text{mat}} = [1.9, 2.0]$: $\mathbf{u}_{\text{mat}}^\top \mathbf{v}_c = 1.9 \cdot 0.5 + 2.0 \cdot 0.6 = 0.95 + 1.2 = 2.15.$, $\exp(2.15) \approx 8.589.$


Bước 2: Tính tổng mẫu số
$$
\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c) = 1.477 + 2.292 + 3.562 + 5.530 + 8.589 \approx 21.450.
$$

Bước 3: Tính xác suất
$$
P(\text{sits} \mid \text{cat}) = \frac{\exp(1.27)}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)} = \frac{3.562}{21.450} \approx 0.166.
$$

##### 5. Tính log-likelihood
$$
\log P(\text{sits} \mid \text{cat}) = \log(0.166) \approx -1.796.
$$

##### 6. Tính hàm mất mát
Hàm mất mát cho cặp từ này là phủ định của log-likelihood:
$$
J = -\log P(\text{sits} \mid \text{cat}) \approx -(-1.796) = 1.796.
$$
##### 7. Tính gradient $\frac{\partial \log P(w_o \mid w_c)}{\partial \mathbf{v}_c}$
Theo công thức gradient:
$$
\frac{\partial \log P(w_o \mid w_c)}{\partial \mathbf{v}_c} = \mathbf{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j.
$$
Bước 1: Tính xác suất $P(w_j \mid w_c)$ cho tất cả từ trong từ vựng

$P(\text{sits} \mid \text{cat}) \approx 0.166$.

$P(\text{the} \mid \text{cat}) = \frac{\exp(0.39)}{21.450} = \frac{1.477}{21.450} \approx 0.069.$

$P(\text{cat} \mid \text{cat}) = \frac{\exp(0.83)}{21.450} = \frac{2.292}{21.450} \approx 0.107.$

$P(\text{on} \mid \text{cat}) = \frac{\exp(1.71)}{21.450} = \frac{5.530}{21.450} \approx 0.258.$

$P(\text{mat} \mid \text{cat}) = \frac{\exp(2.15)}{21.450} = \frac{8.589}{21.450} \approx 0.400.$

Bước 2: Tính kỳ vọng $\sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j$

$$
\begin{align*}
\sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j &= P(\text{the}) \cdot \mathbf{u}_{\text{the}} + P(\text{cat}) \cdot \mathbf{u}_{\text{cat}} + P(\text{sits}) \cdot \mathbf{u}_{\text{sits}} + P(\text{on}) \cdot \mathbf{u}_{\text{on}} + P(\text{mat}) \cdot \mathbf{u}_{\text{mat}} \\
&= 0.069 \cdot [0.3, 0.4] + 0.107 \cdot [0.7, 0.8] + 0.166 \cdot [1.1, 1.2] + 0.258 \cdot [1.5, 1.6] + 0.400 \cdot [1.9, 2.0].
\end{align*}
$$


Tính từng thành phần:

$0.069 \cdot [0.3, 0.4] = [0.0207, 0.0276]$.

$0.107 \cdot [0.7, 0.8] = [0.0749, 0.0856]$.

$0.166 \cdot [1.1, 1.2] = [0.1826, 0.1992]$.

$0.258 \cdot [1.5, 1.6] = [0.387, 0.4128]$.

$0.400 \cdot [1.9, 2.0] = [0.76, 0.8]$.

Cộng lại:$[0.0207 + 0.0749 + 0.1826 + 0.387 + 0.76, 0.0276 + 0.0856 + 0.1992 + 0.4128 + 0.8] \approx [1.4252, 1.5252].$

Bước 3: Tính gradient

$$\mathbf{u}_o = [1.1, 1.2] \quad (\text{vì } w_o = \text{sits}).$$

$$\frac{\partial \log P(w_o \mid w_c)}{\partial \mathbf{v}_c} = \mathbf{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j = [1.1, 1.2] - [1.4252, 1.5252] \approx [-0.3252, -0.3252].$$

##### 8. Cập nhật vector $\mathbf{v}_c$ bằng SGD
Công thức cập nhật:
$$
\mathbf{v}_c \leftarrow \mathbf{v}_c + \eta \left( \mathbf{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j \right),
$$

Với $\eta = 0.1$:
$$
\mathbf{v}c = [0.5, 0.6] + 0.1 \cdot [-0.3252, -0.3252] = [0.5 - 0.03252, 0.6 - 0.03252] \approx [0.46748, 0.56748].
$$

Vector $\mathbf{v}_{\text{cat}}$ được cập nhật thành $[0.46748, 0.56748]$.
##### 9. Ý nghĩa của cập nhật

Gradient $[-0.3252, -0.3252]$ cho thấy sự khác biệt giữa vector ngữ cảnh thực tế $\mathbf{u}_{\text{sits}}$ và kỳ vọng của mô hình trên toàn từ vựng.

Cập nhật này điều chỉnh $\mathbf{v}_{\text{cat}}$ để tăng xác suất dự đoán "sits" trong tương lai.
Quá trình này sẽ được lặp lại cho các cặp từ trung tâm và từ ngữ cảnh khác trong tập dữ liệu, dần dần cải thiện các vector từ.

##### 10. Hàm mất mát tổng
Nếu xét toàn bộ chuỗi với $m = 1$, các cặp từ trung tâm và từ ngữ cảnh là:

$t = 1$: "the" → ngữ cảnh: "cat".
$t = 2$: "cat" → ngữ cảnh: "the", "sits".
$t = 3$: "sits" → ngữ cảnh: "cat", "on".
$t = 4$: "on" → ngữ cảnh: "sits", "mat".
$t = 5$: "mat" → ngữ cảnh: "on".

Tổng log-likelihood là:
$$
\log L = \sum_{t=1}^T \sum_{-m \leq j \leq m, j \neq 0} \log P(w^{(t+j)} \mid w^{(t)}).
$$

Hàm mất mát:
$$
J = -\log L = -\sum_{t=1}^T \sum_{-m \leq j \leq m, j \neq 0} \log P(w^{(t+j)} \mid w^{(t)}).
$$

Trong SGD, ta thường chỉ tính cho một mini-batch (như cặp "cat" và "sits" ở trên). Để tính toàn bộ $J$, ta cần lặp lại các bước trên cho tất cả các cặp và lấy tổng.


#### 2.2.3.5. Implementation

In [ ]:
import numpy as np

# 1. Thiết lập bài toán
# Chuỗi văn bản: "the cat sits on mat"
text = ['the', 'cat', 'sits', 'on', 'mat']
vocab = list(set(text))  # ['the', 'cat', 'sits', 'on', 'mat']
vocab_size = len(vocab)  # 5 từ
vector_dim = 2  # Kích thước vector là 2
learning_rate = 0.1  # Tốc độ học (eta)
window_size = 1  # Kích thước cửa sổ m = 1
epochs = 3  # Số vòng lặp huấn luyện

# 2. Khởi tạo vector từ trung tâm (v_i) và từ ngữ cảnh (u_i)
# Dựa trên ví dụ số học
center_vectors = {
    'the': np.array([0.1, 0.2]),
    'cat': np.array([0.5, 0.6]),
    'sits': np.array([0.9, 1.0]),
    'on': np.array([1.3, 1.4]),
    'mat': np.array([1.7, 1.8])
}

context_vectors = {
    'the': np.array([0.3, 0.4]),
    'cat': np.array([0.7, 0.8]),
    'sits': np.array([1.1, 1.2]),
    'on': np.array([1.5, 1.6]),
    'mat': np.array([1.9, 2.0])
}

# 3. Hàm tính xác suất P(w_o | w_c) bằng softmax
def compute_softmax_prob(center_vector, context_vector, all_context_vectors):
    score = np.dot(context_vector, center_vector)
    exp_score = np.exp(score)
    denominator = sum(np.exp(np.dot(all_context_vectors[word], center_vector)) for word in vocab)
    prob = exp_score / denominator
    return prob

# 4. Hàm tính gradient: ∂ log P(w_o | w_c) / ∂ v_c
def compute_gradient(center_vector, context_vector, all_context_vectors):
    u_o = context_vector
    expected_u = np.zeros(vector_dim)
    denominator = sum(np.exp(np.dot(all_context_vectors[word], center_vector)) for word in vocab)

    for word in vocab:
        u_j = all_context_vectors[word]
        prob_j = np.exp(np.dot(u_j, center_vector)) / denominator
        expected_u += prob_j * u_j

    gradient = u_o - expected_u
    return gradient

# 5. Hàm dự đoán từ ngữ cảnh
def predict_context_words(center_word, center_vectors, context_vectors, top_k=2):
    if center_word not in vocab:
        print(f"Word '{center_word}' not in vocabulary!")
        return []

    v_c = center_vectors[center_word]
    probs = []

    # Tính xác suất P(w_o | w_c) cho tất cả từ trong từ vựng
    for context_word in vocab:
        u_o = context_vectors[context_word]
        prob = compute_softmax_prob(v_c, u_o, context_vectors)
        probs.append((context_word, prob))

    # Sắp xếp theo xác suất giảm dần và lấy top-k
    probs.sort(key=lambda x: x[1], reverse=True)
    return probs[:top_k]

# 6. Hàm huấn luyện
def train_skipgram(text, center_vectors, context_vectors, window_size, epochs, learning_rate):
    for epoch in range(epochs):
        print(f"\n=== Epoch {epoch + 1} ===")
        total_loss = 0

        # Lặp qua từng từ trung tâm trong chuỗi
        for t in range(len(text)):
            center_word = text[t]
            v_c = center_vectors[center_word]

            # Xác định các vị trí ngữ cảnh trong cửa sổ
            context_indices = []
            for j in range(-window_size, window_size + 1):
                if j != 0 and 0 <= t + j < len(text):
                    context_indices.append(t + j)

            # Lặp qua từng từ ngữ cảnh
            for ctx_idx in context_indices:
                context_word = text[ctx_idx]
                u_o = context_vectors[context_word]

                print(f"\nProcessing pair: center = '{center_word}', context = '{context_word}'")
                print(f"Initial v_{center_word} = {v_c}")

                # Tính xác suất P(w_o | w_c)
                prob = compute_softmax_prob(v_c, u_o, context_vectors)
                print(f"P({context_word} | {center_word}) = {prob:.3f}")

                # Tính log-likelihood
                log_prob = np.log(prob)
                print(f"log P({context_word} | {center_word}) = {log_prob:.3f}")

                # Tính hàm mất mát cho cặp này
                loss = -log_prob
                total_loss += loss
                print(f"Loss for this pair = {loss:.3f}")

                # Tính gradient
                gradient = compute_gradient(v_c, u_o, context_vectors)
                print(f"Gradient ∂ log P / ∂ v_{center_word} = {gradient}")

                # Cập nhật vector v_c bằng SGD
                v_c_new = v_c + learning_rate * gradient
                print(f"Updated v_{center_word} = {v_c_new}")

                # Gán lại vector đã cập nhật
                center_vectors[center_word] = v_c_new
                v_c = v_c_new  # Cập nhật v_c để sử dụng trong lần lặp tiếp theo

                # In vector context (không cập nhật trong ví dụ này)
                print(f"u_{context_word} (unchanged) = {u_o}")

        print(f"\nTotal loss for epoch {epoch + 1}: {total_loss:.3f}")
        print("\nCurrent center vectors after epoch:")
        for word in vocab:
            print(f"v_{word} = {center_vectors[word]}")

# 7. Chạy huấn luyện
print("Starting training...")
train_skipgram(text, center_vectors, context_vectors, window_size, epochs, learning_rate)

# 8. In kết quả cuối cùng
print("\nFinal center vectors after training:")
for word in vocab:
    print(f"v_{word} = {center_vectors[word]}")

# 9. Dự đoán từ ngữ cảnh cho từ "cat"
print("\nPredicting context words for 'cat':")
predictions = predict_context_words('cat', center_vectors, context_vectors, top_k=2)
for word, prob in predictions:
    print(f"Context word: '{word}', Probability: {prob:.3f}")

Starting training...

=== Epoch 1 ===

Processing pair: center = 'the', context = 'cat'
Initial v_the = [0.1 0.2]
P(cat | the) = 0.175
log P(cat | the) = -1.744
Loss for this pair = 1.744
Gradient ∂ log P / ∂ v_the = [-0.49540606 -0.49540606]
Updated v_the = [0.05045939 0.15045939]
u_cat (unchanged) = [0.7 0.8]

Processing pair: center = 'cat', context = 'the'
Initial v_cat = [0.5 0.6]
P(the | cat) = 0.069
log P(the | cat) = -2.675
Loss for this pair = 2.675
Gradient ∂ log P / ∂ v_cat = [-1.1255103 -1.1255103]
Updated v_cat = [0.38744897 0.48744897]
u_the (unchanged) = [0.3 0.4]

Processing pair: center = 'cat', context = 'sits'
Initial v_cat = [0.38744897 0.48744897]
P(sits | cat) = 0.177
log P(sits | cat) = -1.729
Loss for this pair = 1.729
Gradient ∂ log P / ∂ v_cat = [-0.26611694 -0.26611694]
Updated v_cat = [0.36083728 0.46083728]
u_sits (unchanged) = [1.1 1.2]

Processing pair: center = 'sits', context = 'cat'
Initial v_sits = [0.9 1. ]
P(cat | sits) = 0.056
log P(cat | sits) = -



## 2.3. Continuous Bag of Words (CBOW)
### 2.3.1. Định nghĩa

CBOW là một kiến trúc học từ biểu diễn (word embedding) do nhóm Google giới thiệu trong mô hình Word2Vec.

Trái ngược với mô hình skip-gam, CBOW giả định rằng một từ có thể được sinh ra bằng cách xử dụng các từ xung quanh nó trong một chuỗi văn bản. Tiếp tục sử dụng ví dụ của skip-gram, giả sử chuỗi văn bản là “the”, “man”, “loves”, “his” và “son”. Ta đặt kích thước cửa sổ ngữ cảnh bằng 2 với (“the”, “man”, “his” và “son”) làm các từ ngữ cảnh nằm. Với các từ ngữ cảnh (“the”, “man”, “his” và “son”), mô hình CBOW quan tâm đến xác suất có điều kiện sinh ra các từ đích trung tâm (“loves”):

$$
P(\textrm{"loves"}\mid\textrm{"the"},\textrm{"man"},\textrm{"his"},\textrm{"son"}).
$$

Trong mô hình CBOW, mỗi từ được biểu diễn bằng hai vector $d-\text{chiều}$ (một dùng khi từ $w$ là từ ngữ cảnh, một dùng khi từ $w$ là từ trung tâm) để tính xác suất có điều kiện. Giả sử chỉ số của một từ trong từ điển là $i$, vector của từ được biểu diễn là $\mathbf{v}_i\in\mathbb{R}^d$ khi từ này là từ ngữ cảnh và là $\mathbf{u}_i\in\mathbb{R}^d$ khi từ này là một từ đích trung tâm (ngược với skip-gram). Gọi $c$ và $o$ lần lượt là chỉ số của từ đích trung tâm $w_c$ và từ các ngữ cảnh ${w}_{o_1}, ..., {w}_{o_{2m}}$ trong từ điển. Có thể thu được xác suất có điều kiện sinh ra từ đích trung tâm bằng các từ ngữ cảnh cho trước bằng phép toán softmax trên tích vô hướng của vector:

$$
P(w_c \mid w_{o_1}, \ldots, w_{o_{2m}}) = \frac{\exp\left(\frac{1}{2m}\mathbf{u}_c^\top (\mathbf{v}_{o_1} + \ldots + \mathbf{v}_{o_{2m}}) \right)}{ \sum_{i \in \mathcal{V}} \exp\left(\frac{1}{2m}\mathbf{u}_i^\top (\mathbf{v}_{o_1} + \ldots + \mathbf{v}_{o_{2m}}) \right)}
$$

Trong đó:
- $w_c$: Từ trung tâm (center word), ví dụ "sat" trong câu "the cat sat on mat".
- ${w}_{o_1}, ..., {w}_{o_{2m}}$: Các từ ngữ cảnh (context word), ví dụ "the", "cat, "on", "mat" trong cùng câu.
- $\mathbf{u}_c \in \mathbb{R}^d$: Vector biểu diễn của từ trung tâm $w_c$ (output vector).
- $\mathbf{v}_{o_1}, ..., \mathbf{v}_{o_{2m}} \in \mathbb{R}^d$: Vector biểu diễn của các từ ngữ cảnh ${w}_{o_1}, ..., {w}_{o_{2m}}$ (input vector).
- $\mathcal{V}$: Tập hợp tất cả từ trong từ điển (vocabulary). Tập chỉ số trong bộ từ vựng là $\mathcal{V} = \{0, 1, \ldots, |\mathcal{V}|-1\}$.
- $\mathbf{u}_i \in \mathbb{R}^d$: Vector biểu diễn của từ $i$ trong từ điển.
- $\exp$: Hàm mũ, $\exp(x) = e^x$
- $\mathbf{u}_c^\top \bar{\mathbf{v}}_o$: Tích vô hướng (dot product) giữa vector $\mathbf{u}_o$ và vector cửa sổ trung bình $\bar{\mathbf{v}}_o$, đo lường mức độ tương đồng giữa từ trung tâm và các từ ngữ cảnh.
- $\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \bar{\mathbf{v}}_o)$: Tổng chuẩn hóa (normalization term) giữa các điểm tương đồng mũ giữa từ các từ ngữ cảnh và từ trung tâm trên toàn bộ từ điển $\mathcal{V}$, đảm bảo các xác suất cộng lại bằng 1.

Để rút gọn, ký hiệu $\mathcal{W}_o= \{w_{o_1}, \ldots, w_{o_{2m}}\}$ và $\bar{\mathbf{v}}_o = \left(\mathbf{v}_{o_1} + \ldots + \mathbf{v}_{o_{2m}} \right)/(2m)$

Khi đó, phương trình trên được đơn giản hóa thành:
$$
P(w_c \mid \mathcal{W}_o) = \frac{\exp\left(\mathbf{u}_c^\top \bar{\mathbf{v}}_o\right)}{\sum_{i \in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)}.
$$

### 2.3.2. Hàm hợp lý (likelihood)
Giả sử ta có một chuỗi văn bản với độ dài $T$, trong đó từ tại vị trí $t$ được ký hiệu là $w^{(t)}$. Với $m$ là kích thước cửa sổ, hàm hợp lý (likelihood) của mô hình là xác suất sinh ra bất kỳ từ đích trung tâm nào dựa vào những từ ngữ cảnh,

Hàm hợp lý được định nghĩa như sau:

$$
L = \prod_{t=1}^{T}  P(w^{(t)} \mid  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}).
$$

Ý nghĩa của hàm hợp lý:
- Hàm $L$ biểu thị xác suất tổng quát để mô hình CBOW sinh ra toàn bộ các từ trung tâm dựa vào những từ ngữ cảnh trong tập dữ liệu.
- Mục tiêu của MLE là tìm các tham số ($\mathbf{v}_i$, $\mathbf{u}_i$) sao cho $L$ đạt giá trị lớn nhất, tức là mô hình dự đoán các từ trung tâm chính xác nhất có thể.

### 2.3.3. Training
Tương tự như việc huấn luyện mô hình Skip-gram, ta sẽ cực tiểu hóa hàm mất mát, và sử dụng [**Stochastic Gradient Descent (SGD)**](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) để huấn luyện mô hình CBOW.

#### 2.3.2.1 Tính toán gradient:
Sau khi đã có được hàm hợp lý của mô hình CBOW là:
$$
L = \prod_{t=1}^{T}  P(w^{(t)} \mid  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}).
$$

Nhiệm vụ của chúng ta là đi tìm ước lượng hợp lý cực đại của mô hình CBOW bằng cách cực tiểu hóa hàm mất mát (tương tự như Skip-gram).<br><br>
Để việc tối ưu diễn ra thuận tiện hơn, ta lấy logarith của hàm hợp lý để biến tích thành tổng:
$$
\log (L) = \sum_{t=1}^{T} \log P\left(w^{(t)} \mid w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}\right).
$$

Sau đó lấy phủ định để có được hàm mất mát:
$$
J = -\log (L) = -\sum_{t=1}^T  \textrm{log}\, P(w^{(t)} \mid  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}).
$$

Lưu ý rằng, khi lấy logarith của phương trình **xác suất có điều kiện sinh ra từ đích trung tâm dựa vào các từ ngữ cảnh cho trước** ta sẽ có:
$$
\begin{align*}
\log\,P(w_c \mid \mathcal{W}_o)
&= \log \frac{\exp\left(\mathbf{u}_c^\top \bar{\mathbf{v}}_o\right)}{\sum_{i \in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)}\\
&= \log\,\left( \exp\left(\mathbf{u}_c^\top \bar{\mathbf{v}}_o\right)\right) - \log\,\left(\sum_{i \in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)\right)\\
&= \mathbf{u}_c^\top \bar{\mathbf{v}}_o - \log\,\left(\sum_{i \in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)\right).
\end{align*}
$$

#### 2.3.2.2. Chứng minh gradient theo $\mathbf{v}_{o_i}$
Và để tìm gradient của hàm mất mát, ta lấy đạo hàm của phương trình trên theo từng vector ngữ cảnh $\mathbf{v}_{o_i}\ (i = 1, \ldots, 2m)$:
$$
\frac{\partial \log\, P(w_c \mid \mathcal{W}_o)}{\partial \mathbf{v}_{o_i}} = \frac{\partial}{\partial \mathbf{v}_{o_i}} \left( \mathbf{u}_c^{\top} \bar{\mathbf{v}}_o \right) - \frac{\partial}{\partial \mathbf{v}_{o_i}} \left( \log \left( \sum_{j \in \mathcal{V}} \exp(\mathbf{u}_j^{\top} \bar{\mathbf{v}}_o) \right) \right)
$$

Hãy tính hạng tử đầu tiên:
$$
\frac{\partial}{\partial \mathbf{v}_{o_i}} \left( \mathbf{u}_c^{\top} \bar{\mathbf{v}}_o \right) = \frac{1}{2m} \mathbf{u}_c^{\top} \frac{\partial}{\partial \mathbf{v}_{o_i}} \left(\sum_{i=1}^{2m}\mathbf{v}_{o_i} \right) = \frac{1}{2m} \mathbf{u}_c
$$

Và hạng tử thứ hai:
$$
\frac{\partial}{\partial \mathbf{v}_{o_i}} \left( \log \left( \sum_{j \in \mathcal{V}} \exp(\mathbf{u}_j^{\top} \bar{\mathbf{v}}_o) \right) \right)
= \frac{1}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^{\top} \bar{\mathbf{v}}_o)} \left( \frac{\partial}{\partial \mathbf{v}_{o_i}} \left( \sum_{j \in \mathcal{V}} \exp(\mathbf{u}_j^{\top} \bar{\mathbf{v}}_o) \right) \right)
$$

Với:
$$
\frac{\partial}{\partial \mathbf{v}_{o_i}} \exp(\mathbf{u}_j^{\top} \bar{\mathbf{v}}_o) = \exp(\mathbf{u}_j^{\top} \bar{\mathbf{v}}_o) \frac{\partial}{\partial \mathbf{v}_{o_i}} \left( \mathbf{u}_j^{\top} \bar{\mathbf{v}}_o \right) = \exp(\mathbf{u}_j^{\top} \bar{\mathbf{v}}_o) \frac{1}{2m} \mathbf{u}_j
$$

Ghép vào hạng tử thứ hai ta có:
$$
\frac{1}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^{\top} \bar{\mathbf{v}}_o)} \left( \sum_{j \in \mathcal{V}}\exp(\mathbf{u}_j^{\top} \bar{\mathbf{v}}_o) \frac{1}{2m} \mathbf{u}_j \right) = \frac{1}{2m} \sum_{j \in \mathcal{V}} \frac{\exp(\mathbf{u}_j^{\top} \vec{v}_o)}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^{\top} \vec{v}_o)} \mathbf{u}_j
$$

Kết hợp các hạng tử, ta có phương trình gradient tổng quát:
$$
\begin{align*}
\frac{\partial \log\, P(w_c \mid \mathcal{W}_o)}{\partial \mathbf{v}_{o_i}}
&= \frac{1}{2m} \mathbf{u}_c - \frac{1}{2m} \sum_{j \in \mathcal{V}} \frac{\exp(\mathbf{u}_j^{\top} \vec{v}_o)}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^{\top} \vec{v}_o)} \mathbf{u}_j \\
&= \frac{1}{2m} \left( \mathbf{u}_c - \sum_{j \in \mathcal{V}} P(w_j \, | \, \mathcal{W}_o) \mathbf{u}_j \right)
\end{align*}
$$

**Ý nghĩa**: Gradient này bao gồm hai phần:
- $\mathbf{u}_c$: Vector từ trung tâm thực tế, đại diện cho từ $w_c$ mà mô hình cần dự đoán.
- $\sum_{j \in \mathcal{V}} P(w_j \mid \mathcal{W}_o) \mathbf{u}_j$: Kỳ vọng của vector từ trung tâm trên toàn bộ từ điển, dựa trên xác suất dự đoán từ đó với ngữ cảnh là các từ nằm trong cửa sổ cho trước.

Gradient này được sử dụng để cập nhật $\mathbf{v}_{o_i}$ trong SGD theo công thức:
$$
\mathbf{v}_{o_i} \leftarrow \mathbf{v}_{o_i} + \eta \,. \frac{1}{2m} \left( \mathbf{u}_c - \sum_{j \in \mathcal{V}} P(w_j \, | \, \mathcal{W}_o) \mathbf{u}_j \right),
$$
với $\eta$ là tốc độ học (learning rate).

#### 2.3.2.3. Kết quả sau huấn luyện

Sau khi huấn luyện, với mỗi từ $w_i$ trong từ điển (có chỉ số $i$), ta thu được hai vector:

- $\mathbf{v}_i$: Vector từ ngữ cảnh.
- $\mathbf{u}_i$: Vector từ đích trung tâm.

Trong các ứng dụng NLP (như phân loại văn bản, phân tích cảm xúc), **vector từ đích trung tâm $\mathbf{u}_i$** thường được sử dụng làm biểu diễn chính cho từ $w_i$, trong mô hình CBOW nó được tối ưu hóa để phản ánh chính xác thông tin ngữ cảnh bao quanh $w_i$.

# 3. Bài tập

## 3.1. Độ phức tạp tính toán của mỗi gradient là bao nhiêu? Nếu từ điển chứa một lượng lớn các từ, điều này sẽ gây ra vấn đề gì?
### 3.1.1. Độ phức tạp tính toán của mỗi gradient

Nhắc lại công thức tính gradient của hàm mất mát theo $\mathbf{v}_c$ trong mô hình Skip-gram (15.1.8):

$$
\frac{\partial \textrm{log}\, P(w_o \mid w_c)}{\partial \mathbf{v}_c} = \mathbf{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j
$$

Nhắc lại công thức tính gradient của hàm mất mát theo $\mathbf{v}_{o_i}$ trong mô hình CBOW (15.1.15):

$$
\frac{\partial \log\, P(w_c \mid \mathcal{W}_o)}{\partial \mathbf{v}_{o_i}} = \frac{1}{2m}\left(\mathbf{u}_c - \sum_{j \in \mathcal{V}} P(w_j \mid \mathcal{W}_o) \mathbf{u}_j \right)
$$


- Dễ thấy, đối với cả 2 mô hình, để tính được gradient ta phải tính xác suất có điều kiện cho mọi từ $w_j$ có trong từ điển $\mathcal{V}$ với một từ cho trước.
- Do đó, độ phức tạp tính toán của mỗi gradient trong cả 2 mô hình đều là $O(|\mathcal{V}| \cdot d)$, với $d$ là số chiều của vector embedding và $\mathcal{V}$ là từ điển


### 3.1.2. Vấn đề khi từ điển chứa một lượng lớn các từ

- Từ độ phức tạp trên ta thấy, mỗi lần huấn luyện cho một cặp từ, ta đều phải tính xác suất cho tất cả các từ trong từ điển
- Điều này rất tốn thời gian, dẫn đến huấn luyện rất chậm hội tụ, không khả thi với dữ liệu thực tế.

**Giải pháp:** Dùng kỹ thuật Negative Sampling (xem mục 15.2.1)

## 3.2. Có một số cụm từ cố định trong tiếng Anh bao gồm nhiều từ, chẳng hạn như “new york”. Bạn sẽ huấn luyện các vector từ của chúng như thế nào? Gợi ý: Xem phần 4 trong bài báo Word2vec[2].

**Ý tưởng: Nhận diện và tạo cụm từ (phrases) trước khi huấn luyện Word2Vec**

Trước khi huấn luyện Word2Vec, ta xác định ra các cụm từ phổ biến trong một ngữ cảnh nhưng ít phổ biến trong các ngữ cảnh khác, như: `"new york"`, `"san francisco"`... (trong khi cụm `"this is"` thì xuất hiện trong rất nhiều ngữ cảnh), sau đó nối chúng lại và coi chúng là một từ riêng trong từ điển khi huấn luyện embedding.

**Phương pháp phát hiện cụm từ: Dựa trên thống kê đồng xuất hiện**
Có nhiều phương pháp phát hiện cụm từ đã được nghiên cứu trước đó, nhưng chúng không nằm trong phạm vi nghiên cứu của bài báo nên nhóm tác giả chỉ sử dụng phương pháp đơn giản và đủ hiệu quả.

Ta kiểm tra xem cặp từ $w_i, w_j$ có nên được nối thành một cụm hay không bằng công thức sau:

$$
\text{score}(w_i, w_j) = \frac{C(w_i w_j) - \delta}{C(w_i) \cdot C(w_j)}
$$

Trong đó:
- $C(w_i)$, $C(w_j)$: tần suất của từng từ riêng lẻ,
- $C(w_i w_j)$: tần suất của cụm 2 từ xuất hiện liên tiếp,
- $\delta$: một hằng số để tránh nối những cụm có tần suất thấp (thường là 5).

Đồng thời đặt ra một ngưỡng cố định. Nếu score vượt ngưỡng này, ta xem $w_i w_j$ là cụm cố định.

> **Ví dụ 1**:  
> Nếu `"new"` xuất hiện 5000 lần, `"york"` 3000 lần, `"new york"` xuất hiện 2800 lần, thì:
>
> $\text{score}(\text{"new"}, \text{"york"}) = \frac{2800 - 5}{5000 \cdot 3000} \approx 0.000186$
>
> Nếu ngưỡng là 0.0001 → `"new york"` được giữ lại thành `"new_york"`.

> **Ví dụ 2**:  
> Nếu `"this"` xuất hiện 10000 lần, `"is"` 9500 lần, `"this is"` xuất hiện 5000 lần, thì:
>
> $\text{score}(\text{"this"}, \text{"is"}) = \frac{5000 - 5}{10000 \cdot 9500} \approx 0.0000525789474$
>
> Nếu ngưỡng là 0.0001 → `"this is"` không được nối lại thành cụm từ

## 3.3. Sử dụng mô hình skip-gam làm ví dụ để tìm hiểu về thiết kế của mô hình word2vec. Mối quan hệ giữa tích vô hướng của hai vector từ và độ tương tự cô-sin trong mô hình skip-gam là gì? Đối với một cặp từ có ngữ nghĩa gần nhau, tại sao hai vector từ này lại thường có độ tương tự cô-sin cao?

### 3.3.1. Mối quan hệ giữa tích vô hướng của hai vector từ và độ tương tự cô-sin

Nhắc lại công thức tính vô hướng trong trường hợp 2 vector từ:

$$
\boldsymbol{u}_{w_o}^\top \boldsymbol{v}_{w_c} = \|\boldsymbol{u}_{w_o}\| \cdot \|\boldsymbol{v}_{w_c}\| \cdot \cos(\theta)
$$

Trong đó:
- $\theta$: góc giữa hai vector.
- $\cos(\theta)$: **độ tương tự cosine** giữa hai vector, có miền giá trị từ -1 (khi 2 vector ngược hướng) đến 1 (khi 2 vector cùng hướng) và bằng 0 khi 2 vector vuông góc

=> **Tích vô hướng càng lớn (theo chiều dương)**, nghĩa là:
- Hai vector càng **cùng hướng** (cosine gần 1),
- Hoặc có **độ dài lớn** (cũng ảnh hưởng, nhưng không mang nghĩa ngữ nghĩa nhiều bằng góc).



### 3.3.2. Vì sao các từ có nghĩa gần nhau thường có độ tương tự cosine cao?

Nhắc lại công thức của Skip-gram: Với cặp từ $(w_c, w_o)$, ta tối đa hóa xác suất sau:

$$
P(w_o | w_c) = \frac{\exp(\boldsymbol{u}_{w_o}^\top \boldsymbol{v}_{w_c})}{\sum_{w \in V} \exp(\boldsymbol{u}_w^\top \boldsymbol{v}_{w_c})}
$$

Quan sát công thức này ta thấy: Nếu hai từ hay cùng xuất hiện, tức là xác suất $P$ ở trên lớn, thì tích vô hướng $\boldsymbol{u}_{w_o}^\top \boldsymbol{v}_{w_c}$ cũng lớn.

Mặt khác, ta huấn luyện Skip-gram bằng cách cực đại hoá hợp lý xác suất $P$ đó. Do vậy, mô hình đã được huấn luyện sao cho làm tăng tích vô hướng của 2 vector từ tương tự nhau, từ đó làm tăng độ tương tự cosine.


**Tóm lại:**
- Trong mô hình Skip-Gram của Word2Vec, xác suất dự đoán từ phụ thuộc vào tích vô hướng giữa hai vector từ, mà tích vô hướng lại phụ thuộc trực tiếp vào độ tương tự cosine.
- Khi các từ có nghĩa gần nhau, chúng thường xuất hiện trong những ngữ cảnh giống nhau, khiến các vector của chúng hướng gần nhau trong không gian → độ tương tự cosine cao.